# Run model again, but this time using 'Standardized Enrollment' as the target variable

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt


c:\Users\kathr\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Read csvs
enrollment_df = pd.read_csv("./part-00000-65e9defc-9b69-4b81-a0a8-f3f99a80aaab-c000.csv")
final_data_df = pd.read_csv("./FinalData.csv")

In [3]:
# Drop the non-beneficial ID columns
final_data_df = final_data_df.drop(columns= ['Contract Number', 'Organization Type', 'Contract Name', 'Organization Marketing Name'])
final_data_df

,Parent Organization,Year,C Breast Cancer Screening,C Breast Cancer Screening Non-numeric,C Colorectal Cancer Screening,C Colorectal Cancer Screening Non-numeric,C Cardiovascular Care – Cholesterol Screening,C Cardiovascular Care – Cholesterol Screening Non-numeric,C Diabetes Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening Non-numeric,...,D Statin Use in Persons with Diabetes (SUPD) Non-numeric,C Controlling High Blood Pressure,C Controlling High Blood Pressure Non-numeric,C Transitions of Care,C Transitions of Care Non-numeric,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions Non-numeric,Overall,Overall Non-numeric,Standardized Enrollment
0,Centene Corporation,2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured,1
1,Humana Inc.,2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured,1
2,"UnitedHealth Group, Inc.",2014,2.0,Numeric,2.0,Numeric,4.0,Numeric,4.0,Numeric,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,3.0,Numeric,1
3,BlueCross BlueShield of Alabama,2014,3.0,Numeric,4.0,Numeric,5.0,Numeric,4.0,Numeric,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,3.0,Numeric,2
4,"BLUE CROSS AND BLUE SHIELD OF MONTANA, INC.",2014,0.0,Plan too new to be measured,0.0,Plan too new to be measured,0.0,Plan too new to be measured,0.0,Plan too new to be measured,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Plan too new to be measured,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4425,Rite Aid Corporation,2020,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Numeric,0.0,Numeric,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable,2
4426,Capital BlueCross,2020,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Numeric,0.0,Numeric,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable,1
4427,Anthem Inc.,2020,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Numeric,0.0,Numeric,...,Plan too new to be measured,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable,1
4428,"UnitedHealth Group, Inc.",2020,0.0,Plan not required to report measure,0.0,Plan not required to report measure,0.0,Numeric,0.0,Numeric,...,Numeric,0.0,Numeric,0.0,Numeric,0.0,Numeric,0.0,Not Applicable,2


In [4]:
# Determine the number of unique values in each column.
final_data_df.nunique()

Parent Organization                                                                                               364
Year                                                                                                                7
C Breast Cancer Screening                                                                                           6
C Breast Cancer Screening Non-numeric                                                                               5
C Colorectal Cancer Screening                                                                                       6
                                                                                                                 ... 
C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions                  1
C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions Non-numeric      1
Overall                                                 

In [6]:
# Select featuers
feature_cols = ['Year', 'C Breast Cancer Screening', 'C Colorectal Cancer Screening', 'C Cardiovascular Care – Cholesterol Screening', 'C Diabetes Care – Cholesterol Screening', 'C Glaucoma Testing', 'C Annual Flu Vaccine',
                 'C Improving or Maintaining Physical Health', 'C Improving or Maintaining Mental Health', 'C Monitoring Physical Activity', 'C Adult BMI Assessment', 'C Care for Older Adults – Medication Review',
                   'C Care for Older Adults – Functional Status Assessment', 'C Care for Older Adults – Pain Screening', 'C Osteoporosis Management in Women who had a Fracture', 'C Diabetes Care – Eye Exam', 
                   'C Diabetes Care – Kidney Disease Monitoring', 'C Diabetes Care – Blood Sugar Controlled', 'C Diabetes Care – Cholesterol Controlled', 'C Controlling Blood Pressure', 'C Rheumatoid Arthritis Management',
                     'C Improving Bladder Control', 'C Reducing the Risk of Falling', 'C Plan All-Cause Readmissions', 'C Getting Needed Care', 'C Getting Appointments and Care Quickly', 'C Customer Service', 
                     'C Rating of Health Care Quality', 'C Rating of Health Plan', 'C Care Coordination', 'C Complaints about the Health Plan', 'C Beneficiary Access and Performance Problems', 'C Members Choosing to Leave the Plan',
                       'C Health Plan Quality Improvement', 'C Plan Makes Timely Decisions about Appeals', 'C Reviewing Appeals Decisions', 'C Call Center – Foreign Language Interpreter and TTY Availability',
                         'HD1: Staying Healthy: Screenings, Tests and Vaccines', 'HD2: Managing Chronic (Long Term) Conditions', 'HD3: Member Experience with Health Plan', 
                         "HD4: Member Complaints, Problems Getting Services, and Improvement in the Health Plan's Performance", 'HD5: Health Plan Customer Service', 'SNP', 'Part C Summary', 
                         'D Call Center – Foreign Language Interpreter and TTY Availability', 'D Appeals Auto–Forward', 'D Appeals Upheld', 'D Complaints about the Drug Plan', 'D Beneficiary Access and Performance Problems',
                           'D Members Choosing to Leave the Plan', 'D Drug Plan Quality Improvement', 'D Rating of Drug Plan', 'D Getting Needed Prescription Drugs', 'D MPF Price Accuracy', 'D High Risk Medication', 'D Diabetes Treatment',
                             'D Medication Adherence for Diabetes Medications ', 'D Medication Adherence for Hypertension (RAS antagonists) ', 'D Medication Adherence for Cholesterol (Statins) ', 'DD1: Drug Plan Customer Service', 
                             'DD2: Member Complaints, Problems Getting Services, and Improvement in the Drug Plan’s Performance ', 'DD3: Member Experience with the Drug Plan', 'DD4: Patient Safety and Accuracy of Drug Pricing', 
                             'Part D Summary', 'C Special Needs Plan (SNP) Care Management', 'C Care for Older Adults – Pain Assessment', "HD4: Member Complaints and Changes in the Health Plan's Performance", 
                             'DD2: Member Complaints and Changes in the Drug Plan’s Performance', 'DD4: Drug Safety and Accuracy of Drug Pricing', 'D Medication Adherence for Diabetes Medications', 
                             'D Medication Adherence for Hypertension (RAS antagonists)', 'D Medication Adherence for Cholesterol (Statins)', 'D MTM Program Completion Rate for CMR', 'C Medication Reconciliation Post-Discharge', 
                             'C Statin Therapy for Patients with Cardiovascular Disease', 'D Statin Use in Persons with Diabetes (SUPD)', 'C Controlling High Blood Pressure', 'C Transitions of Care', 
                             'C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions', 'Overall']

In [7]:
# Show feature vs target variable
y = final_data_df["Standardized Enrollment"]
X = final_data_df[feature_cols]

In [8]:
# Use scalar to normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# Split data in to test and training data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [11]:
# Train the data using the Logistic Regression Model
model = LogisticRegression()
model.fit(X_train, y_train)

c:\Users\kathr\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_probability = model.predict_proba(X_test)[:, 1]

In [ ]:
# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 0.7348
Classification Report:
               precision    recall  f1-score   support

           1       0.79      0.87      0.82       491
           2       0.67      0.65      0.66       336
           3       0.44      0.14      0.22        49
           4       0.20      0.10      0.13        10

    accuracy                           0.73       886
   macro avg       0.52      0.44      0.46       886
weighted avg       0.72      0.73      0.72       886



#### Feature Importance

In [15]:
# Train a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X, y)

RandomForestClassifier(random_state=42)

In [16]:
# Use Random Forest in sklearn to auto calculate feature importance
importances = rf_model.feature_importances_

In [17]:
# Sort features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.05436427777245295, 'C Rheumatoid Arthritis Management'),
 (0.039749243809387526, 'D Members Choosing to Leave the Plan'),
 (0.03694279536034697,
  'C Osteoporosis Management in Women who had a Fracture'),
 (0.028717758447888805, 'D Appeals Upheld'),
 (0.02431756155809582, 'D Rating of Drug Plan'),
 (0.023910434924680306, 'C Reviewing Appeals Decisions'),
 (0.022813288045809812, 'HD5: Health Plan Customer Service'),
 (0.02157063749542199, 'DD3: Member Experience with the Drug Plan'),
 (0.02153711094165575, 'C Plan Makes Timely Decisions about Appeals'),
 (0.020842207553340337, 'C Reducing the Risk of Falling'),
 (0.019737168763656106, 'D Getting Needed Prescription Drugs'),
 (0.0196416299756576, 'D Complaints about the Drug Plan'),
 (0.019221774099327172, 'C Rating of Health Care Quality'),
 (0.01744775222348211, 'C Diabetes Care – Blood Sugar Controlled'),
 (0.016937631929683065, 'C Improving or Maintaining Physical Health'),
 (0.01692053402204159, 'C Care Coordination'),
 (0.01681